In [3]:
# Sample Dataset
# Format: each row is an example.
# The last column is the label.
# The first two columns are features.
training_data = [
    ["Green", 3, "Mange"],
    ["Yellow", 3, "Mange"],
    ["Red", 1, "Grape"],
    ["Red", 1, "Grape"],
    ["Yellow", 3, "Lemon"]
]

# Column labels.
header = ["color", "diameter", "label"]

def unique_vals(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    counts = {}
    for row in rows:
        # In our dataset format, the label is always the last colum.
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

def is_numeric(value):
    """Test if a value is numeric."""
    return isinstance(value, int) or isinstance(value, float)

class Question:
    """A Question is used to partition a dataset.
    
    this class just records a 'column number' (e.g., 0 for color) and a
    'column value' (e.g., Green). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question.
    """
    
    def __init__(self, column, value):
        self.column = column
        self.value = value
    
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value
    
    def __repr__(self):
        # this is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
        header[self.column], condition, str(self.value))

def partition(rows, question):
    """Partitions a dataset.
    
    For each row in the dataset, check if it matches the question if 
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

def gini(rows):
    """Calculate the Gini impurity for a list of rows."""
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl]/float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

def info_gain(left, right, current_uncertainity):
    """Information Gain.
    
    The uncertainity of the starting node, minus the weighted impurity of
    two child nodes.
    Eq. IG(feature) = -P(yes) * log2(P(yes)) - P(no) * log2(P(no))
    """
    p = float(len(left))/(len(left) + len(right))
    return current_uncertainity - p * gini(left) - (1 - p) * gini(right)

def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value 
    and calculating the information gain."""
    best_gain = 0 # keep track of the best information gain
    best_question = None # keep train of the feature / value that produced it.
    current_uncertainity = gini(rows)
    n_features = len(rows[0]) - 1 # number of columns
    
    for col in range(n_features): # for each feature
        values = unique_vals(rows, col) # unique values in the column
        
        for val in values: # for each value
            question = Question(col, val)
            
            # try spliting the dataset.
            true_rows, false_rows = partition(rows, question)
            
            # Skip this split if it doesn't devide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            
            # calculate the information gain for this split
            gain = info_gain(true_rows, false_rows, current_uncertainity)
            
            if gain > best_gain:
                best_gain, best_question = gain, question
    
    return best_gain, best_question

class Leaf:
    """A Leaf node classifies data.
    
    This holds a dictionary of class (e.g., Mango) -> number of times
    it apears in the rows from the training data that reach this leaf
    """
    
    def __init__(self, rows):
        self.predictions = class_counts(rows)

class Decision_Node:
    """A Decision Node asks a question.
    
    This holds a reference to the question, and to the two child nodes.
    """
    
    def __init__(self,
                question,
                true_brances,
                false_branches):
        self.question = question
        self.true_branches = true_brances
        self.false_branches = false_branches

def build_tree(rows):
    """Builds thr tree."""
    
    # Try partitionaing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain
    gain, question = find_best_split(rows)
    
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)
    
    # if we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch.
    true_branch = build_tree(true_rows)
    
    # Recursively build the false branch.
    false_branch = build_tree(false_rows)
    
    # Returna Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # depending on the answer.
    return Decision_Node(question, true_branch, false_branch)


def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""
    
    # Base case: we've reached a leaf.
    if isinstance(node, Leaf):
        print(spacing + "Predict", node.predictions)
        return
    
    # print the question at this node.
    print(spacing + str(node.question))
    
    # Call this function recursively on the true branch
    print(spacing + "--> True:")
    print_tree(node.true_branches, spacing=" ")
    
    # Call this function recursively on the false branch
    print(spacing + "--> False:")
    print_tree(node.false_branches, spacing=" ")
    
def classify(row, node):
    
    # Base case: we've reached a leaf.
    if isinstance(node, Leaf):
        return node.predictions
    
    # Decide whether to follow the true-branch or the false-branch,
    # Compare the feature / value stored in the node,
    # to the example we're considering
    if node.question.match(row):
        return classify(row, node.true_branches)
    else:
        return classify(row, node.false_branches)

def print_leaf(counts):
    """Print the predictions at a leaf."""    
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl]/total * 100)) + "%"
    return probs

In [5]:
if __name__ == "__main__":
    my_tree = build_tree(training_data)
    
    print_tree(my_tree)
    
    # Evaluate
    testing_data = [
        ["Green", 3, "Mango"],
        ["Yellow", 4, "Mango"],
        ["Red", 2, "Grape"],
        ["Red", 1, "Grape"],
        ["Yellow", 3, "Lemon"]
    ]
    
    for row in testing_data:
        print("Actual: %s, Predicted: %s" %
             (row[-1], print_leaf(classify(row, my_tree))))

Is color == Red?
--> True:
 Predict {'Grape': 2}
--> False:
 Is color == Yellow?
 --> True:
 Predict {'Mange': 1, 'Lemon': 1}
 --> False:
 Predict {'Mange': 1}
Actual: Mango, Predicted: {'Mange': '100%'}
Actual: Mango, Predicted: {'Mange': '50%', 'Lemon': '50%'}
Actual: Grape, Predicted: {'Grape': '100%'}
Actual: Grape, Predicted: {'Grape': '100%'}
Actual: Lemon, Predicted: {'Mange': '50%', 'Lemon': '50%'}
